A practical introduction to Pandas
===================

We have been asked to *analyze an otp dataset*, without much more information. This kind of scenario is more common than you might imagine!

## 1) Quickly examine the files in ~/Data/us_dot/otp. What do they contain, in both technical and functional terms? (Use any tool you want)

In [3]:
path_to_zips = '../../data/us_dot/otp/'

For this example, we are going to begin using the shell from within this notebook, with the ! notation:

A ! sign before a line tells the notebook to send that line straight away to the underlying OS. 

\* Note that we can substitute python variables into the shell command. We do that by surrounding the name of the variable with curly braces ({}). That's what we are going to do with the `path_to_files` variable

### Check what the files contain

In [4]:
! ls {path_to_zips} 

ls: cannot access '../../data/us_dot/otp/': No such file or directory


In [5]:
! unzip -l {path_to_zips}On_Time_On_Time_Performance_2015_1.zip

unzip:  cannot find or open ../../data/us_dot/otp/On_Time_On_Time_Performance_2015_1.zip, ../../data/us_dot/otp/On_Time_On_Time_Performance_2015_1.zip.zip or ../../data/us_dot/otp/On_Time_On_Time_Performance_2015_1.zip.ZIP.


Oh, it has a readme! Always good to read it.

### Unzip the readme to the current directory

In [6]:
! unzip -o {path_to_zips}On_Time_On_Time_Performance_2015_1.zip readme.html -d $(pwd)

unzip:  cannot find or open ../../data/us_dot/otp/On_Time_On_Time_Performance_2015_1.zip, ../../data/us_dot/otp/On_Time_On_Time_Performance_2015_1.zip.zip or ../../data/us_dot/otp/On_Time_On_Time_Performance_2015_1.zip.ZIP.


In [7]:
! cat readme.html | head

cat: readme.html: No such file or directory


The readme file is html. Luckily, we are working in an html environment. 

### Display the contents of `readme.html` within the notebook
(Hint: check out [IPython.display.IFrame](https://ipython.org/ipython-doc/3/api/generated/IPython.display.html))

In [8]:
from IPython.display import IFrame
IFrame('readme.html', width=700, height=350)

That's some very good documentation!

### Summary: 

The files within the zip are " quoted csv's. They contain information on timeliness of departures in the US, at the departure level.

In [9]:
# have a look at the beginning of one of the files to see what they look like

! unzip -p {path_to_zips}On_Time_On_Time_Performance_2015_1.zip | head -n 2

Pretty unreadable, so we go for a tool designed specifically for tabular data: **pandas**

### Load some of the data (one of the files) into memory as a pandas dataframe. What functions do you need to use?

Pro tip: there is no need to decompress the whole file.

First, open a connection to one of the files

In [10]:
import os
import pandas as pd

onemonth = os.listdir(path_to_zips)[0]
df = pd.read_csv(path_to_zips + onemonth, compression='zip')

FileNotFoundError: [Errno 2] No such file or directory: '../../data/us_dot/otp/'

In [ ]:
import zipfile

zip_file = zipfile.ZipFile(path_to_zips + onemonth)

zip_file is a connection to the compressed file, the .zip. We can use it to open a connection to one of the files it contains, which will behave like a normal uncompressed file that we had opened with open()

In [ ]:
csv, readme = zip_file.filelist

csv_file = zip_file.open(csv.filename)

Now we're ready to load the file into memory as a pandas dataframe. Remember to close the connections to the files!

In [12]:
# We reopen the file because we have already consumed the header line.
# We could also seek(0)
import pandas as pd

csv_file = zip_file.open(csv.filename)
df = pd.read_csv(csv_file)

csv_file.close()
zip_file.close()

NameError: name 'zip_file' is not defined

Let's start examining the data: show the beginning of the file. How many records does it contain?

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
print(df.size)
df.size == df.shape[0] * df.shape[1]

In [ ]:
df.dtypes

## Trimming the data

The table is quite wide, and it seems that there are many columns without much data. Which, exactly, are those? (let's consider empty a column that doesn't contain at least 1000 records, arbitrarily)

In [11]:
df.count()

NameError: name 'df' is not defined

In [ ]:
non_null_counts = df.count()

non_null_counts[non_null_counts < 1000]

It seems that the "diverted" fields, after the first, are often empty. No big surprise, since not that many flights must be diverted more than once in a month. Let's drop those columns, since we are not that interested in those, at least for now.

In [ ]:
# Dropping inplace leads to confusion. Confusion leads to fear. 
# Fear leads to anger. Anger leads to hate. Hate leads to suffering.
df2 = df.drop(df.columns[non_null_counts < 1000], axis=1)

In [ ]:
df2.shape

In [ ]:
# Another way to do it

df.dropna(axis=1, thresh=1000).shape == df2.shape

Now that we have eliminated some inconvenient data columns, let's have a look at the rest:

Let's loook at how the location data is encoded.

### Select the columns that have 'Origin' in their name

Hint: we are going to use the str attribute of Series and Indexes

In [ ]:
col_contains_origin = df2.columns.str.contains('Origin')
origin_cols = df2.columns[col_contains_origin]
origin_cols

Let's now have a look at these. We have been using the .head() method, but that always returns the same values, so it's not that good for getting a feel for the data. Let's start using a better one, that will give us a sample of the data: df.sample()

In [ ]:
df2[origin_cols].sample(5)

#### Exercise: 

Show all airport codes in NY city and Washington DC.

In [ ]:
print(df2[df2['OriginCityName'] == 'New York, NY']['Origin'].unique())

print(df2[df2['OriginCityName'].str.contains('Washington')]['Origin'].unique())

So much redundant information!!!!! Let's drop a few of those fields. Remember, 90% of the time spent in Data Science is data cleaning... 

### Do the same for the destination columns

In [ ]:
col_contains_dest = df2.columns.str.contains('Dest')

df2.columns[col_contains_dest]

### Now, discard all these columns with redundant information. 

The set of columns we want to discard is 

`df.columns[col_contains_dest] + df.columns[col_contains_origin] - columns_of_interest`

We'll also use the opportunity to drop Year, Month, Day and Quarter columns since that is information that is already contained in FlightDate. We'll keep the DayOfWeek, though.

In [ ]:
columns_of_interest = pd.Index(['Origin', 'OriginCityName', 'OriginStateName', 'Dest', 'DestCityName', 'DestStateName'])


In [ ]:
# We could build the Index (or list) in a single line, but this way it's more readable:

cols_to_drop = pd.Index(['Year', 'Month', 'Quarter', 'DayofMonth'])

cols_to_drop = cols_to_drop | df2.columns[col_contains_dest]

cols_to_drop = cols_to_drop | df2.columns[col_contains_origin]

cols_to_drop = cols_to_drop.difference(columns_of_interest)

In [ ]:
df3 = df2.drop(cols_to_drop, axis=1)

In [ ]:
df3.sample(5)

OK, so we have cleaned up most of the date / location information,  and some of the extra information that was not really useful, in the process reducing the dataframe from 110 columns to 60. Let's look at the really interesting info: delays.

## Fast forward

All of this is getting a little repetitive and boring, so let's just specify the columns we want:

In [ ]:
df = df[['FlightDate', 'DayOfWeek', 'Carrier', 'TailNum', 'FlightNum', 'Origin', 
         'OriginCityName', 'OriginStateName', 'Dest', 'DestCityName', 'DestStateName',
         'DepTime', 'DepDelay', 'AirTime', 'Distance']]
df.sample(5)

## Formatting columns and parsing dates and times

Hurray! we have almost cleaned our dataset. Soon we will begin to do some actual work with it. 

In [ ]:
df.dtypes

Pandas interpreted the Deptime column as ints and the FlighDate column as strings. We want to combine them and parse them into a DateTime column, so that we can use them properly as datetimes.

First, 

### Define a function that will parse our int hours into a reasonable format ("HH:MM")

In [ ]:
# We'll need to take an int and return a string of the appropriate format

def deptime_to_string(num):
    
    hour = int(num / 100) % 24 # There are 24s in the data, which datetime doesn't like
    minute = int(num % 100)
    

    return '%02d:%02d' % (hour, minute)

deptime_to_string(2423.0)

Now, we can use that function to build datetime strings that we will then pass to pd.to_datetime, with a format we will specify. Let's do that

Hint: Check out [pd.to_datetime's documentatoin](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html) for info on the acceptable format.

### Overwrite the 'DepTime' column with its version in the proper format

Hint: Before overwriting your column in the dataframe, make sure that everything works by assigning the modified column to a Series variable.

In [ ]:
series = df['DepTime'].dropna().apply(deptime_to_string)

df['DepTimeStr'] = series

In [ ]:
# What happened to the nas? They are still nas, because our 
# Series didn't contain them. Therefore, when we put the 
# Series back into the Dataframe, those cells stayed empty.

df['DepTimeStr'].count()

### Now, create a DepDateTime with the proper type using `pd.to_datetime`

In [ ]:
# This is the format pd.to_datetime needs:

ts = pd.to_datetime('2015-01-15 08:30')
ts

In [ ]:
df['DepDateTime'] = departure_datetimes = pd.to_datetime(df['FlightDate'] + ' ' + df['DepTimeStr'])

# Drop the now-redundant columns
df = df.drop(['FlightDate', 'DepTime', 'DepTimeStr'], axis=1)

In [ ]:
df.sample(5)

### Inspect the types, see if everything is in order

In [ ]:
df.dtypes

### Transform the FlightNum column into a column of strings

Because flight numbers are actually names, not numbers: we won't do arithmetic on them. It wouldn't make sense to receive summary statistics on the `FlightNum` column with `.describe()`, for example.

In [ ]:
df['FlightNum'] = df['FlightNum'].map(str)

## Examining the data

Finally clean! Let's start to do some preliminary work on the data.

### Find the biggest delays. 

How would you find the 5 maximum delays?

In [ ]:
df.sort_values(by='DepDelay', ascending=False).head()

### What was the average delay for this month? Standard deviation and typical value?

In [ ]:
df['DepDelay'].describe()

Also: a quick look at the correlation between the numerical variables is extremely easy with pandas:

In [ ]:
df.corr()

### A little taste of groupby

Very often, we will want to split our data according to a variable, then compute some statistics on the different groups. We will see this in depth next week, but I want to give you a little taste.

What is the plane that has the highest average delay? We'll first group by tail number (the *license plate* of a plane) and then calculate the relevant statistic for each group (group of *departures*)

In [ ]:
grouped_by_plane = df.groupby('TailNum')
type(grouped_by_plane)

In [ ]:
grouped_by_plane['DepDelay'].mean().sort_values(ascending=False).head(10)

Oh, but those numbers smell like these planes had only a few, very delayed, departures! how can we count the number of departures *and* calculate the average delay at the same time?

In [ ]:
delays_by_plane = grouped_by_plane['DepDelay'].agg(['mean', 'count'])

In [ ]:
delays_by_plane.sort_values('mean', ascending=False)

Awesome! now, let's look at the average delay of the planes with some departures (let's say, at least 15)

In [ ]:
consistently_delayed = delays_by_plane[delays_by_plane['count'] > 14]
highly_delayed = consistently_delayed.sort_values('mean', ascending=False).head(100)

# What companies do those planes belong to?
df.join(highly_delayed, on='TailNum', how='inner')['Carrier'].unique()

#### Exercise: 

Show cities by descending number of airports

In [ ]:
airports_by_city = df3.groupby('OriginCityName')['Origin'].unique()

airports_by_city.map(lambda x: len(x)).sort_values(ascending=False).head()

## Plotting

There are several ways in which we could go about plotting this dataset in order to get acquainted with it. 

For example, do the delays have a relationship with the number of departures a plane does?

In [ ]:
# Do a scatter plot with matplotlib. Check the documentation

import matplotlib.pyplot as plt 
%matplotlib inline

ax = delays_by_plane.plot.scatter('mean', 'count', alpha=.1)
ax.set_xlim(-20,240)
ax.set_ylim(-10,240)

Pretty impossible to see anything in there. Maybe a different kind of plottting is required:

In [ ]:
ax = delays_by_plane.plot.hexbin('mean', 'count')
ax.set_xlim(-20,240)
ax.set_ylim(-10,240)

What is the distribution of delays?

In [ ]:
# Little trick: Just by importing seaborn, it will change 
# matplotlib defaults and make your graphs much prettier.

import seaborn as sns

ax = df['DepDelay'].hist(bins=500)
ax.set_xlim(-100, 500)

plt.title('Number of flights per delay bin')

In [ ]:
# We can make any axis logarithmic in order to see better a very unequal distribution

ax = df['DepDelay'].hist(bins=500)
ax.set_xlim(-100, 500)
plt.title('Num flights per delay bin')

plt.yscale('log')


### Another question:

how do the delays stack over the course of the day? We are going to look at it by plotting the distribution of delays for each hour of the day. The very best way to compare distributions side by side is a boxplot, so we'll use that.

In [ ]:
help(plt.boxplot)

We need to provide `plt.boxplot()` with a sequence that contains 24 elements. Each of those will be a sequence containing every individual delay for one hour of the day. We'll need, therefore, to extract hours of the day for each departure and group based on that.

In [ ]:
# Now, we can iterate over the groups and extract the delays. 
# We keep them in two different vectors because that is what boxplot wants

df['DepHour'] =  df['DepDateTime'].map(lambda x: x.hour)

by_hour = df.groupby('DepHour')

hours = []
groups = []

for h, g in by_hour:
    hours.append("%02d" % h)
    groups.append(g['DepDelay'])

In [ ]:
# We have create a groupby object. That object behaves like a list.
# Each element of that list is a (key, dataframe) tuple.
# You can think of the key as the group's name. The dataframe
# contains all the rows in the original dataframe that correspond 
# to that key: Here, group '0.0' contains all rows from flights 
# that departed on hour 0.

print(type(by_hour))
print(list(by_hour.head(10)))
print(list(by_hour)[0], list(by_hour)[1])

### Everything looks ok! let's plot this thing!

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 6))

ax1.set_ylim(-20, 240)

bp = plt.boxplot(groups)

labels = plt.setp(ax1, xticklabels=hours)